# Production Scheduling Example Using Pulp

In [99]:
#Importing Necessary Libraries
import pandas as pd 
from pulp import *

In [98]:
#Read the Excel file containing production data
param = pd.read_excel('Production_assignment_python.xlsx')


In [81]:
#Rename the coulumn without the heading
param = param.rename(columns = {'Unnamed:0': 'Period'})

In [82]:
param

,period,demand,fixed cost,produuction Cost,storage cost,Capacity
0,period 1,100,2000,40,1,4000
1,period 2,4000,2000,50,1,6000
2,period 3,2500,2000,80,1,4000
3,period 4,4000,2000,60,1,12000
4,period 5,6000,2000,50,1,4000
5,period 6,100,2000,60,1,4000
6,period 7,250,3000,20,1,8000
7,period 8,5000,3000,40,1,4000
8,period 9,6000,3000,90,1,4000
9,period 10,4000,3000,30,1,7000


In [83]:
param['t'] = range(1,13)



In [84]:
param = param.set_index('t')

In [85]:
param

,period,demand,fixed cost,produuction Cost,storage cost,Capacity
t,,,,,,
1,period 1,100,2000,40,1,4000
2,period 2,4000,2000,50,1,6000
3,period 3,2500,2000,80,1,4000
4,period 4,4000,2000,60,1,12000
5,period 5,6000,2000,50,1,4000
6,period 6,100,2000,60,1,4000
7,period 7,250,3000,20,1,8000
8,period 8,5000,3000,40,1,4000
9,period 9,6000,3000,90,1,4000


In [86]:
inventory = LpVariable.dicts("Inv",[0,1,2,3,4,5,6,7,8,9,10,11,12],0,None,'Integer')
inventory[0] = 0
production = LpVariable.dicts("Prod",[1,2,3,4,5,6,7,8,9,10,11,12],0,None,'Integer')
Binary = LpVariable.dicts("Bin",[1,2,3,4,5,6,7,8,9,10,11,12],0,None,'Binary')

In [87]:
time = [1,2,3,4,5,6,7,8,9,10,11,12]

In [88]:
 model = LpProblem('Production',LpMinimize)

In [89]:
model += lpSum([inventory[t]*param.loc[t,'storage cost'] + production[t]*param.loc[t,'produuction Cost']+ Binary[t]*
                param.loc[t,'fixed cost'] for t in time])

In [94]:
for t in time:
    model+= production[t] - inventory[t]+ inventory[t-1] >= param.loc[t,'demand']
    model+= Binary[t] * param.loc[t,'Capacity'] production[t]

In [95]:
model.solve()

1

In [96]:
optimization_values = pd.DataFrame({'Demand': param['demand'],
                                   'Production':[production[i].varValue for i in production],
                                   'Inventory': [inventory[i].varValue for i in range(1,13)],
                                   'Opening':[Binary[i].varValue for i in Binary] })

In [97]:
optimization_values

,Demand,Production,Inventory,Opening
t,,,,
1,100,4000.0,3900.0,1.0
2,4000,6000.0,5900.0,1.0
3,2500,0.0,3400.0,0.0
4,4000,2600.0,2000.0,1.0
5,6000,4000.0,0.0,1.0
6,100,1250.0,1150.0,1.0
7,250,8000.0,8900.0,1.0
8,5000,4000.0,7900.0,1.0
9,6000,0.0,1900.0,0.0
